<br></br>
# **Project4 | word2vec**
<br></br>
## **1 데이터 전처리**

In [ ]:
# 살인의 추억 텍스트 불러오기
filename = './data/movie_memories_of_murder_2003.txt'
with open(filename, 'r') as f:
    texts = f.read()
texts[:500]

In [ ]:
from txtutil import txtnoun
skips = {'두만':'박두만', '태윤':'서태윤', '용구':'조용구', '귀옥':'권귀옥', 
         '희봉':'구희봉', '동철':'신동철', '광호':'백광호', '병순':'조병순', 
         '해일':'박해일', '광호의':'백광호', '백광호의':'백광호'}
texts  = txtnoun(filename, skip=skips, tags=['Noun'])
texts[:500]

In [ ]:
# 명사 Token 작업된 자료를 ssResport.txt 로 저장 
script_file = './data/mom_script.txt'
with open(script_file, 'w', encoding='utf-8') as file:
    file.write(texts)
    
print("전처리 완료된 대본파일 저장 : {}".format(script_file))

In [ ]:
# ! cat ./data/ssResport.txt | head -n 10

<br></br>
## **2 Word 2 vec 객체 만들기**
gensim

In [ ]:
# ! pip3 install gensim

In [ ]:
%%time
from gensim.models import word2vec
data  = word2vec.LineSentence(script_file)
model = word2vec.Word2Vec(data, size=30, window=2, min_count=10, hs=1,
                          workers=4, iter=100, sg=1)

model_file = "./data/mom_script.model"
model.save(model_file)
print("Word 2 Vec 모델 파일 {} 저장완료".format(model_file))

<br></br>
## **3 저장된 모델 불러오기 및 확인**
gensim

In [ ]:
%reset

In [ ]:
%who

In [ ]:
model_file = "./data/mom_script.model"

from gensim.models import word2vec
model = word2vec.Word2Vec.load(model_file)
len(model.wv.vocab.keys())

In [ ]:
list(model.wv.index2word)

<br></br>
## **4 Word2Vec 모델 내용 확인**
gensim

In [ ]:
model.wv.most_similar(positive=['범인'])

In [ ]:
model.wv.most_similar(negative=['범인'])

In [ ]:
model.wv.most_similar(positive=['피해자'])

In [ ]:
model.wv.most_similar(negative=['피해자'])

In [ ]:
model.wv.most_similar(positive=['박두만'])

In [ ]:
model.wv.most_similar(positive=['서태윤'])

In [ ]:
model.wv.most_similar(positive=['조용구'])

In [ ]:
model.wv.most_similar(positive=['박두만','서태윤'])

In [ ]:
model.wv.most_similar(positive=['박두만','서태윤'],
                      negative=['피해자'])

In [ ]:
model.wv.most_similar(positive=['박두만','서태윤'],
                      negative=['박해일'])

In [ ]:
model.wv.most_similar(positive=['박두만','서태윤'],
                      negative=['범인'])

<br></br>
## **5 Visulaization**
matplotlib (T-SNE 로 차원축소)

In [ ]:
list(model.wv.vocab.keys())[:10]

In [ ]:
# model.wv.vocab : { word: object of numeric vector }
vocab  = list(model.wv.vocab)
X      = model[vocab]
X.shape

In [ ]:
%%time
from sklearn.manifold import TSNE
tsne   = TSNE(n_components = 2)
X_tsne = tsne.fit_transform(X)

In [ ]:
import pandas as pd
df = pd.DataFrame(X_tsne, 
                  index   = vocab, 
                  columns = ['x', 'y'])
df.head()

In [ ]:
%matplotlib inline
from matplotlib import rc
rc('font', family=['NanumGothic','Malgun Gothic'])

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12,12))
ax  = fig.add_subplot(1, 1, 1)
ax.scatter(df['x'], df['y'])
for word, pos in df.iterrows():
    ax.annotate(word, pos, fontsize=15)
plt.grid(True)